In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0037555694580078125
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 21.63321768059711
200000 31.897302531365415
300000 31.841578161610425
400000 31.041963819242664
500000 32.620240227812346
600000 31.07146295927266
700000 30.510003510580034
800000 30.951661349681793
900000 25.847346535438817
1000000 17.617281714733135
1100000 30.490959898913175
1200000 30.351580401348816
1300000 30.570784564011753
1400000 32.18539905391856
1500000 30.85021981328833
1600000 30.57202023241434
1700000 31.02159943404856
1800000 28.926921883850078
1900000 13.973369640719968
2000000 29.95821816742171
2100000 29.546502134927156
2200000 28.649949454086162
2300000 30.23516047814845
2400000 29.179502764468758
2500000 28.37192080140397
2600000 28.622292588106127
2700000 28.16202301444089
2800000 12.816048674734729
2900000 28.466325995517323
3000000

29400000 22.36601178407957
29500000 31.417950587886704
29600000 29.717305442949797
29700000 30.023146126656357
29800000 28.047099269924903
29900000 27.16963228797752
30000000 12.387547552614196
30100000 27.77579038205835
30200000 27.209937629146356
30300000 29.003561882561677
30400000 29.864525797851993
30500000 30.74987123388216
30600000 30.35684824614223
30700000 15.605224567991407
30800000 29.774110511285294
30900000 30.76479611802888
31000000 29.914139572382936
31100000 29.379657764662785
31200000 15.495636590659132
31300000 30.137816807020833
31400000 29.20987129226321
31500000 29.90390336607321
31600000 27.932762786752278
31700000 25.89789758271248
31800000 15.673196721437018
31900000 23.1384748048185
32000000 25.900199139200634
32100000 28.06495881571905
32200000 28.453313178871536
32300000 29.31871438347393
32400000 30.59624944197233
32500000 15.537995861483605
32600000 28.980990518182637
32700000 29.709115150571503
32800000 30.363364682005876
32900000 31.180211961178006
330000

59100000 7.5624206119950586
59200000 14.57829640887087
59300000 14.76469333870062
59400000 15.76342161972697
59500000 6.816179638876475
59600000 18.023868435920377
59700000 17.2653258040408
59800000 14.060305243305423
59900000 20.104574430683087
60000000 20.838717219440557
60100000 12.537874463538888
60200000 15.744415738902864
60300000 13.342545478834543
60400000 17.009975967263806
60500000 10.122545524667023
60600000 12.888232235308266
60700000 11.545704266539367
60800000 11.8856185363413
60900000 8.89441823950948
61000000 14.74571900532736
61100000 14.920760789011359
61200000 10.887590584583778
61300000 6.410011434294849
61400000 12.209124688578276
61500000 12.901128000041703
61600000 16.930312064292732
61700000 15.229316705619494
61800000 14.979648534007064
61900000 21.759598609368698
62000000 19.16489434060168
62100000 13.974201284636054
62200000 12.309907458401065
62300000 9.518038197360527
62400000 15.49135474890665
62500000 17.13131309612968
62600000 11.943225871263177
62700000

88800000 24.719506637667656
88900000 26.647473167412095
89000000 27.20801659350911
89100000 26.630535330691856
89200000 24.695137284332844
89300000 22.604218874605667
89400000 22.571003768687607
89500000 23.502003665040164
89600000 24.15891701298321
89700000 18.734543516182608
89800000 14.71791693153482
89900000 15.600859353290021
90000000 18.016010496873456
90100000 13.257678624905683
90200000 22.06004127069767
90300000 22.842179960863792
90400000 23.69299492856462
90500000 24.444651426679133
90600000 23.325042748150103
90700000 25.769576178034967
90800000 25.964139410274537
90900000 27.12968989339022
91000000 26.303920358326476
91100000 23.427084091307254
91200000 25.507511740063844
91300000 23.24588407730728
91400000 24.337842852878342
91500000 23.269702010425682
91600000 11.103902603517678
91700000 19.209238776431768
91800000 15.487783052983689
91900000 13.2877400666527
92000000 23.554877065386968
92100000 23.609703630167054
92200000 23.295781969782723
92300000 23.962429642959428
9

117900000 30.869562435398077
118000000 33.34265667947377
118100000 28.587526665314122
118200000 24.029806489371122
118300000 24.95629798728048
118400000 23.429319007896165
118500000 24.43404056118595
118600000 25.86544335332738
118700000 23.31899267811876
118800000 23.010542295689355
118900000 16.066317656679637
119000000 3.505888535893282
119100000 15.85873031432957
119200000 24.416592431891598
119300000 23.432583170135906
119400000 24.687921780916536
119500000 24.452135621300034
119600000 25.514355861038986
119700000 24.420377073673468
119800000 24.7407486371837
119900000 26.43779377215755
120000000 22.080904346046083
120100000 21.58928096049708
120200000 19.520049262464404
120300000 21.708916264377656
120400000 20.81595018312762
120500000 19.985237582595797
120600000 20.042794632028112
120700000 14.077997531869347
120800000 3.9367586479888863
120900000 11.500705353692862
121000000 20.39468193697761
121100000 20.168044922326096
121200000 20.486530318726544
121300000 21.08928234040086

...

320200000 5.272190067393221
320300000 4.997252943590373
320400000 5.107658525825771
320500000 2.2246625153533253
320600000 2.4689628352681297
320700000 4.77281899957041
320800000 5.457267439049511
320900000 5.789957525403837
321000000 5.419980584664034
321100000 5.389921953139772
321200000 5.960844710822433
321300000 5.291659946996748
321400000 5.890934684155971
321500000 5.326912688675503
321600000 2.8774656309601494
321700000 4.3360569086335765
321800000 5.047189990144769
321900000 4.673459022647288
322000000 5.77599794196754
322100000 4.815468669983417
322200000 4.7907905269398094
322300000 3.0313113755738574
322400000 1.6922407552927137
322500000 5.292258782935361
322600000 5.2911044220928485
322700000 5.382134900368621
322800000 5.772971698263586
322900000 5.537952392708253
323000000 5.597370156721383
323100000 5.052357597692944
323200000 6.158121363632957
323300000 5.165944331449926
323400000 2.5177214799201484
323500000 4.793582302370379
323600000 5.376585025134981
323700000 4.8

348400000 3.5580462048042585
348500000 2.7311994372108677
348600000 3.8103738370411557
348700000 3.959579559647865
348800000 3.9595939540116785
348900000 3.1229788109178016
349000000 3.472460380037569
349100000 2.883586602932632
349200000 2.6562659692987665
349300000 3.493502862276223
349400000 2.566620183031837
349500000 3.7199274703743526
349600000 4.188050928431426
349700000 3.91639372763742
349800000 3.078367103309333
349900000 3.442179712146842
350000000 3.217295812832326
350100000 2.454218260930601
350200000 3.4398131662975477
350300000 2.532908352450351
350400000 3.465229970292479
350500000 4.3530708323882195
350600000 3.9445480907616512
350700000 2.8541300547391937
350800000 3.5355604981096524
350900000 3.3931591697984635
351000000 2.417805646706909
351100000 3.502255676633133
351200000 2.7378650640724502
351300000 3.6954111866401695
351400000 4.308584941476319
351500000 3.740491788391978
351600000 3.1672505871414542
351700000 3.221714426807473
351800000 3.521425920536221
35190

377100000 1.4812257420013448
377200000 2.233135533848031
377300000 1.3461340208495853
377400000 2.101221052562634
377500000 1.6591107068172462
377600000 2.0317298474713765
377700000 2.2661248360026875
377800000 2.3071304032030304
377900000 1.8186690316308438
378000000 1.589760137503508
378100000 2.062912850955496
378200000 1.1068500316692491
378300000 2.127355468152977
378400000 1.2793870760272577
378500000 2.083916100713829
378600000 2.265322639832566
378700000 2.2324762528923516
378800000 2.0620435797078316
378900000 1.491529577861084
379000000 1.8927703298001852
379100000 1.2189553612092834
379200000 1.7417776910179423
379300000 1.3260191308496427
379400000 1.77164001209866
379500000 1.479526552660336
379600000 1.9014391951238347
379700000 1.8248498292686641
379800000 1.3782456960056488
379900000 1.7409636024807824
380000000 1.2090770750558772
380100000 1.2557132505197262
380200000 1.3232461295304927
380300000 1.243627822240889
380400000 1.6796439690684577
380500000 1.21806755744679

404300000 0.8618576002275347
404400000 0.45446942429423004
404500000 0.5352268691164156
404600000 0.6447829025251964
404700000 0.8070736860426135
404800000 0.4644361386873424
404900000 0.3552481945691705
405000000 0.7208630393628876
405100000 0.8528494324165484
405200000 0.5621016150734326
405300000 0.29244778782342207
405400000 0.7594060493599442
405500000 0.6109237342973947
405600000 1.0954986508450442
405700000 0.5675817670839628
405800000 0.307918077963382
405900000 0.5410546258303225
406000000 0.6016078167157968
406100000 0.5482951342355807
406200000 0.3437141779693001
406300000 0.5340271428851817
406400000 0.5870415261356433
406500000 0.7663213552240399
406600000 0.5519805636311967
406700000 0.5967900553653539
406800000 1.1084946051260796
406900000 0.7064991396534376
407000000 0.1297573452999592
407100000 0.17757672221014897
407200000 0.12377070036545396
407300000 0.1614720007667343
407400000 0.12168937234132819
407500000 0.12138244335671267
407600000 0.1207373312357962
407700000

430600000 0.11097990193079246
430700000 0.17228417501481375
430800000 0.09516353433415294
430900000 0.07697139279686412
431000000 0.09974299574249983
431100000 0.07042358596434195
431200000 0.09924673475077415
431300000 0.06852354532873631
431400000 0.12020071438056229
431500000 0.09125871783636345
431600000 0.10666135924412808
431700000 0.16082774305768807
431800000 0.10070780226752162
431900000 0.08519940532383323
432000000 0.06158275608876347
432100000 0.08902997420196401
432200000 0.0648047739623785
432300000 0.1085976352930367
432400000 0.12260728431677488
432500000 0.08371464465666811
432600000 0.09580962223279146
432700000 0.0629101107442975
432800000 0.0988568149933219
432900000 0.07352745393627881
433000000 0.07127397590925959
433100000 0.07687033630219102
433200000 0.08763169334108961
433300000 0.05115455062242349
433400000 0.04506911233028769
433500000 0.05561341503740681
433600000 0.04315882302161058
433700000 0.045104755985677245
433800000 0.05426001769820187
433900000 0.0